# 🤪 AEI_NET

In [ ]:
# BEFORE YOU START YOU NEED TO HAVE EXECUTED pretraining.ipynb

In [1]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Set memory growth to avoid pre-allocation of all GPU memory
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

2025-02-11 15:24:17.385364: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-11 15:24:17.446034: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-11 15:24:17.464009: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-11 15:24:18.232722: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but

In [2]:
%load_ext autoreload
%autoreload 2

# Limit TensorFlow to 80% of GPU memory
from gpu_memory import limit_gpu_memory 
#limit_gpu_memory(0.35)
limit_gpu_memory(0.55)

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import optimizers

1 Physical GPUs, 1 Logical GPUs
Memory limit set to 13510.2 MB per GPU


2025-02-11 15:24:29.104318: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-11 15:24:29.105387: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-02-11 15:24:29.105475: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-02-11 15:24:29.105517: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA

In [3]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16') #This was produucing NAN when training beyond ~7 epochs
tf.config.optimizer.set_jit(True)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4090, compute capability 8.9


2025-02-11 15:24:43.579609: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


## 0. Parameters <a name="parameters"></a>

In [4]:
IMAGE_SIZE = 64
CHANNELS = 3
BATCH_SIZE = 32
NUM_FEATURES = 512
Z_DIM = 200
LEARNING_RATE = 1e-4 #0.0005
WEIGHT_DECAY = 1e-7
EPOCHS = 5
BETA = 2000
LOAD_MODEL = True
TAKE_BATCHES = 500
VALIDATION_SET_SIZE = 10

## 1. Prepare the data <a name="prepare"></a>

In [7]:
from data_loader import get_training_data

train, validation = get_training_data()

Take 242


2025-02-11 15:26:11.075258: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [ ]:
((img_batch, embed_batch), (Y_target, Y_target_x2)) = next(iter(train))
train_sample = img_batch.numpy()
result_sample = Y_target.numpy()
result_x2_sample = Y_target_x2.numpy()


In [ ]:
# Show some faces from the training set
from notebooks.utils import display

display(train_sample, 8, cmap=None)
display(result_sample, 8, cmap=None)
display(result_x2_sample, 8, cmap=None)
print(result_sample[0])
print(embed_batch.shape)
print(embed_batch)

## 2. Build the AEI_NET <a name="build"></a>

In [ ]:
from aei_net import get_model
generator = get_model(num_blocks=3)

In [ ]:
from discriminator import MinimalPatchGAN
discriminator = MinimalPatchGAN(ndf=64, num_layers=3)


## 3. Train the AEI_NET <a name="train"></a>

In [ ]:
#from tensorflow_addons.optimizers import AdamW
#optimizer = AdamW(
#    weight_decay=WEIGHT_DECAY,
#    learning_rate=LEARNING_RATE)
##model.compile(optimizer=optimizer, loss=['mse', None])  
#model.compile(optimizer=optimizer, loss=['mae', None])  

In [ ]:
from training_callbacks import get_callbacks
model_checkpoint_callback, tensorboard_callback, image_generator = get_callbacks(train)
terminate_on_nan = tf.keras.callbacks.TerminateOnNaN()

class FinalModelSaver(tf.keras.callbacks.Callback):
    def __init__(self, save_path):
        super().__init__()
        self.save_path = save_path

    def on_epoch_end(self, epoch, logs=None):
        """
        Save the model after each epoch.
        """
        save_path_with_epoch = f"{self.save_path}_epoch_{epoch + 1}"
        self.model.save(save_path_with_epoch)
        print(f"Model saved at: {save_path_with_epoch}")
        self.model.save(f"{self.save_path}_latest")
        print(f"Model saved at: {self.save_path}_latest")

In [ ]:
from AEIGANModel import AEIGANModel
aei_gan_model = AEIGANModel(generator=generator, 
                            discriminator=discriminator,
                            lambda_recon=150.0,
                            lambda_adv=0.5,
                            lambda_id=45.0
                           )

from tensorflow_addons.optimizers import AdamW
aei_gan_model.compile(
    g_optimizer=AdamW(learning_rate=4e-4, beta_1=0, beta_2=0.999, weight_decay=1e-4), 
    d_optimizer=AdamW(learning_rate=4e-6, beta_1=0, beta_2=0.999, weight_decay=1e-4), 
)

In [ ]:
from tensorflow_addons.optimizers import AdamW

# Load old weights if required
if LOAD_MODEL:
    aei_gan_model.load_weights("./models/aei_net_gan_kitchensink_128_v3_150_0.5_45_15_latest")
    aei_gan_model.g_optimizer=AdamW(learning_rate=4e-4, beta_1=0, beta_2=0.999, weight_decay=1e-4) 
    aei_gan_model.d_optimizer=AdamW(learning_rate=4e-6, beta_1=0, beta_2=0.999, weight_decay=1e-4)
    tmp = aei_gan_model.predict(train.take(1))

In [ ]:
dummy_img = tf.zeros([1, 64, 64, 3], dtype=tf.float32)
dummy_embed = tf.zeros([1, 512], dtype=tf.float32)
aei_gan_model((dummy_img, dummy_embed), training=False)

In [ ]:
history = aei_gan_model.fit(
    train,
    validation_data=validation,  # optional
    epochs=EPOCHS,
    callbacks=[
        FinalModelSaver("./models/aei_net_gan_kitchensink_128_v3_150_0.5_45_15"),
        terminate_on_nan,
        model_checkpoint_callback,
        # tensorboard_callback,
        image_generator,
    ]
)


In [ ]:
from super_resolution import SuperResModel
superres_model = SuperResModel(aei_gan_model.generator)
superres_model.compile(
    g_optimizer=AdamW(learning_rate=4e-4, beta_1=0, beta_2=0.999, weight_decay=1e-4), 
)

In [ ]:
from tensorflow_addons.optimizers import AdamW

# Load old weights if required
if LOAD_MODEL:
    superres_model.load_weights("./models/superres_aei_net_gan_kitchensink_128_v3_150_0.5_45_15_latest")
    superres_model.g_optimizer=AdamW(learning_rate=4e-4, beta_1=0, beta_2=0.999, weight_decay=1e-4) 
    tmp = superres_model.predict(train.take(1))

In [ ]:
dummy_img = tf.zeros([1, 64, 64, 3], dtype=tf.float32)
dummy_embed = tf.zeros([1, 512], dtype=tf.float32)
superres_model((dummy_img, dummy_embed), training=False)

In [ ]:
superres_model.fit(
    train, 
    epochs=EPOCHS, 
    callbacks=[
        FinalModelSaver("./models/superres_aei_net_gan_kitchensink_128_v3_150_0.5_45_15"),
        terminate_on_nan,
        model_checkpoint_callback,
        # tensorboard_callback,
        image_generator,
    ]
)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=logs

## 3. Reconstruct using the variational autoencoder <a name="reconstruct"></a>

In [ ]:

for layer in generator.layers:
    print(layer.name, layer.input_shape)
    if layer.name == "model_1":
        for l in layer.layers:
            print(l.name, l.input_shape)


In [ ]:
# Select a subset of the test set
((img_batch, embed_batch), (Y_target, Y_target_x2)) = next(iter(train.skip(10)))
example_images = img_batch.numpy()
example_embed = embed_batch.numpy()
example_target = Y_target.numpy()
example_target_x2 = Y_target_x2.numpy()

In [ ]:
# Create autoencoder predictions and display
Y_pred = aei_gan_model([example_images, example_embed])
y_upsampled = superres_model([example_images, example_embed])

predicted_images = np.clip(Y_pred * 255, 0, 255).astype(np.uint8)
predicted_images_x2 = np.clip(y_upsampled * 255, 0, 255).astype(np.uint8)
print("Example real faces")
display(example_images)
print("Reconstructions")
display(predicted_images)
print("Reconstructions x2")
display(predicted_images_x2)
print("Target")
display(example_target)
print("Target x2")
display(example_target_x2)

In [ ]:
from inference import main as infer
models = [
    (3, "aei_net_gan_weights_latest", "superres_weights_superres_latest")
]
for num_blocks, m, s in models:
    infer(
        model_paths = (f"./models/{m}", f"./models/{s}"),
        source_image_path = "./samples/1.jpg", 
        target_image_path = "./samples/2.jpg",
        output_file_name=f"{m}-o1.png",
        num_blocks=num_blocks
    )
    infer(
        model_paths = (f"./models/{m}", f"./models/{s}"),
        source_image_path = "./samples/3.jpg", 
        target_image_path = "./samples/2.jpg",
        output_file_name=f"{m}-o2.png",
        num_blocks=num_blocks
    )
    infer(
        model_paths = (f"./models/{m}", f"./models/{s}"),
        source_image_path = "./samples/4.jpg", 
        target_image_path = "./samples/2.jpg",
        output_file_name=f"{m}-o3.png",
        num_blocks=num_blocks
    )
    infer(
        model_paths = (f"./models/{m}", f"./models/{s}"),
        source_image_path = "./samples/generated_img_004_6.png", 
        target_image_path = "./samples/2.jpg",
        output_file_name=f"{m}-o4.png",
        num_blocks=num_blocks
    )
    infer(
        model_paths = (f"./models/{m}", f"./models/{s}"),
        source_image_path = "./samples/generated_img_005_8.png", 
        target_image_path = "./samples/2.jpg",
        output_file_name=f"{m}-o4.png",
        num_blocks=num_blocks
    )

In [ ]:
clear_cached_dataset()

In [ ]:
from super_resolution import SuperResModel
model = SuperResModel(aei_gan_model.generator)

for layer in model.layers:
    print(layer.name, layer.input_shape)

In [8]:
%reset -f